Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [461]:
from random import choices,randint

import lab9_lib

In [462]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

00111111000101001001110111110110010101111001101011: 9.11%
01110010101001101111010100000010001000011010001001: 23.56%
00000101110101011011010000101000100000101000011100: 7.34%
10100101110001111011111011001011001000011110111100: 23.33%
11110000000011110110111101010110000101110111011001: 23.33%
00100000111100101000011011000111100101010011000011: 15.36%
00000111101001111011011101000110010100111111110111: 9.11%
01011111111110101100000100110000001100110110000110: 7.33%
00011110001101000100001110110110100111011100011101: 15.33%
11010001000111111101000001100101000000010100101101: 15.34%
10


## EA

In [463]:
import random


def mutation(genome):
    index = randint(0,len(genome[0])-1)
    genome[0][index] = 1-genome[0][index]
    return genome[0]


def one_cut_xover(ind1, ind2):
    cut_point = randint(0, len(ind1[0]))
    offspring = ind1[0][:cut_point]+ind2[0][cut_point:]
    return offspring

def xover(genome1, genome2):
    child_genome = [g1 if random.random() > 0.5 else g2 for g1, g2 in zip(genome1[0], genome2[0])]
    return child_genome

In [464]:
from random import choice


def select_parent(population,fitness):
    pool = [choice(population) for _ in range(2)]
    champion = max(pool, key=lambda i: i[1])
    return champion

In [465]:
def init_population(n_individual,length,fitness):
    pop = []
    for _ in range(n_individual):
        ind = (choices([0, 1], k=length))
        pop.append((ind,fitness(ind)))
    return pop

In [466]:
def gen_new_population(n_new_indivdual,mutation_prob,old_population,fitness):
    new_individual = []
    for _ in range(n_new_indivdual):
        if random.random() < mutation_prob:
            old_ind = select_parent(old_population,fitness)
            tmp = mutation(old_ind)
        else:
            old_ind = select_parent(old_population,fitness)
            old_ind_2 = select_parent(old_population,fitness)
            tmp = xover(old_ind,old_ind_2)
        new_individual.append((tmp,fitness(tmp)))
    return new_individual

In [467]:
def replacement(new_pop,old_pop,fitness):
    tmp_pop = new_pop + old_pop
    sorted_pop = sorted(tmp_pop,key= lambda i:i[1],reverse=True)
    return sorted_pop[:len(new_pop)]

In [468]:
N_INDV = 500
LENGTH_INDV = 1000
GENERATION = 200
fit = lab9_lib.make_problem(5)
pop = init_population(N_INDV,LENGTH_INDV,fit)

for g in range(GENERATION):
    if g%10 == 0:
        print(pop[0][1])
    new_pop = gen_new_population(N_INDV,0.1,pop,fit)
    pop = replacement(new_pop,pop,fit)
    
print(fit.calls)

0.090239
0.4147
0.42210000000000003
0.42219999999999996
0.42610000000000003
0.4342
0.4342
0.4342
0.4342
0.4342
0.437
0.437
0.437
0.437
0.4465
0.53
0.53
0.53
0.53
0.53
100500
